In [1]:
import starfile, numpy as np, pandas as pd

def inspect_star(path):
    df = starfile.read(path)
    print("\n===", path, "===")
    print("rows:", len(df))
    print("cols:", df.columns.tolist())

    # common coordinate column sets:
    for cols in [
        ["rlnCoordinateX","rlnCoordinateY","rlnCoordinateZ"],
        ["rlnCenteredCoordinateXAngst","rlnCenteredCoordinateYAngst","rlnCenteredCoordinateZAngst"],
        ["rlnCenteredCoordinateX","rlnCenteredCoordinateY","rlnCenteredCoordinateZ"],
    ]:
        if all(c in df.columns for c in cols):
            arr = df[cols].to_numpy(float)
            print("using cols:", cols)
            print("min:", arr.min(axis=0), "max:", arr.max(axis=0))
            break
    return df

df_pytom = inspect_star("reconstruction_fixed_particles_pytom.star")
df_ri    = inspect_star("reconstruction_fixed_particles_ri.star")
df_std   = inspect_star("reconstruction_fixed_particles_std.star")


=== reconstruction_fixed_particles_pytom.star ===
rows: 137
cols: ['rlnCenteredCoordinateXAngst', 'rlnCenteredCoordinateYAngst', 'rlnCenteredCoordinateZAngst', 'rlnAngleRot', 'rlnAngleTilt', 'rlnAnglePsi', 'rlnLCCmax', 'rlnCutOff', 'rlnSearchStd', 'rlnTomoTiltSeriesPixelSize', 'rlnTomoName']
using cols: ['rlnCenteredCoordinateXAngst', 'rlnCenteredCoordinateYAngst', 'rlnCenteredCoordinateZAngst']
min: [-3465. -3540. -1245.] max: [3600. 3690. -900.]

=== reconstruction_fixed_particles_ri.star ===
rows: 150
cols: ['rlnCenteredCoordinateXAngst', 'rlnCenteredCoordinateYAngst', 'rlnCenteredCoordinateZAngst', 'rlnAngleRot', 'rlnAngleTilt', 'rlnAnglePsi', 'rlnLCCmax', 'rlnCutOff', 'rlnSearchStd', 'rlnTomoTiltSeriesPixelSize', 'rlnTomoName']
using cols: ['rlnCenteredCoordinateXAngst', 'rlnCenteredCoordinateYAngst', 'rlnCenteredCoordinateZAngst']
min: [-3075. -3540. -1125.] max: [3375. 3375. 1155.]

=== reconstruction_fixed_particles_std.star ===
rows: 150
cols: ['rlnCenteredCoordinateXAngst', 

In [2]:
import mrcfile

with mrcfile.open("reconstruction.mrc", permissive=True) as m:
    print("Shape:", m.data.shape)
    print("Voxel size:", m.voxel_size)

/workspaces/fle3d-experiment1/.venv/lib/python3.10/site-packages/mrcfile/mrcinterpreter.py:206: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
/workspaces/fle3d-experiment1/.venv/lib/python3.10/site-packages/mrcfile/mrcinterpreter.py:216: RuntimeWarning: Unrecognised machine stamp: 0x00 0x00 0x00 0x00
  warnings.warn(str(err), RuntimeWarning)


Shape: (512, 512, 512)
Voxel size: (nan, nan, nan)


/workspaces/fle3d-experiment1/.venv/lib/python3.10/site-packages/mrcfile/mrcobject.py:354: RuntimeWarning: invalid value encountered in divide
  x = self.header.cella.x / self.header.mx
/workspaces/fle3d-experiment1/.venv/lib/python3.10/site-packages/mrcfile/mrcobject.py:355: RuntimeWarning: invalid value encountered in divide
  y = self.header.cella.y / self.header.my
/workspaces/fle3d-experiment1/.venv/lib/python3.10/site-packages/mrcfile/mrcobject.py:356: RuntimeWarning: invalid value encountered in divide
  z = self.header.cella.z / self.header.mz


In [3]:
import starfile
import numpy as np

VOL_SHAPE = 512
PIX_A = 15.0

def star_to_voxel(path):
    df = starfile.read(path)
    xyz_A = df[[
        "rlnCenteredCoordinateXAngst",
        "rlnCenteredCoordinateYAngst",
        "rlnCenteredCoordinateZAngst"
    ]].values

    # Convert Å → voxel and shift center
    xyz_vox = xyz_A / PIX_A + VOL_SHAPE / 2
    return xyz_vox

coords_pytom = star_to_voxel("reconstruction_fixed_particles_pytom.star")
coords_ri    = star_to_voxel("reconstruction_fixed_particles_ri.star")
coords_std   = star_to_voxel("reconstruction_fixed_particles_std.star")

In [4]:
import pandas as pd

gt_df = pd.read_csv("particle_locations.txt", sep=" ", header=None)
gt_xyz = gt_df.iloc[:, 1:4].values   # columns 1,2,3

In [5]:
gt_xyz_xyz = gt_xyz
gt_xyz_zyx = gt_xyz[:, ::-1]

In [7]:
print(gt_df["label"].value_counts())

label
5MRC          131
1QVR          129
4CR2          119
4V94_fixed    118
1U6G          115
2CG9          112
3H84          111
3D2F          110
3QM1          108
1S3X          104
3GL1          104
3CF3          102
1BXN           88
fiducial       10
vesicle         5
Name: count, dtype: int64


In [8]:
gt_5mrc = gt_df[gt_df["label"] == "5MRC"]

print("GT 5MRC count:", len(gt_5mrc))

gt_xyz_raw = gt_5mrc[["c1","c2","c3"]].to_numpy(float)

print("GT min:", gt_xyz_raw.min(axis=0))
print("GT max:", gt_xyz_raw.max(axis=0))

GT 5MRC count: 131
GT min: [22. 19. 21.]
GT max: [491. 487. 160.]


In [16]:
import starfile
import numpy as np

# Load star files
pytom_df = starfile.read("reconstruction_fixed_particles_pytom.star")
ri_df    = starfile.read("reconstruction_fixed_particles_ri.star")
std_df   = starfile.read("reconstruction_fixed_particles_std.star")

# Extract Angstrom coordinates
cols = [
    "rlnCenteredCoordinateXAngst",
    "rlnCenteredCoordinateYAngst",
    "rlnCenteredCoordinateZAngst"
]

pytom_xyz_ang = pytom_df[cols].to_numpy(float)
ri_xyz_ang    = ri_df[cols].to_numpy(float)
std_xyz_ang   = std_df[cols].to_numpy(float)

print("Loaded:")
print("PyTom:", pytom_xyz_ang.shape)
print("RI:", ri_xyz_ang.shape)
print("STD:", std_xyz_ang.shape)

Loaded:
PyTom: (137, 3)
RI: (150, 3)
STD: (150, 3)


In [17]:
voxel_size = 15.0  # because reconstruction is 15Å

pytom_xyz_vox = (pytom_xyz_ang / voxel_size) + 256
ri_xyz_vox    = (ri_xyz_ang / voxel_size) + 256
std_xyz_vox   = (std_xyz_ang / voxel_size) + 256

In [18]:
pytom_xyz_vox

array([[ 60., 403., 189.],
       [ 57., 408., 185.],
       [ 59., 402., 185.],
       [ 61., 497., 195.],
       [ 59., 407., 189.],
       [260.,  62., 194.],
       [213., 203., 180.],
       [389.,  20., 176.],
       [408., 405., 175.],
       [137., 419., 177.],
       [261., 325., 185.],
       [496., 345., 196.],
       [ 66., 343., 183.],
       [102., 363., 182.],
       [405., 170., 181.],
       [216., 440., 174.],
       [145., 358., 179.],
       [345., 247., 196.],
       [287., 250., 188.],
       [226.,  95., 175.],
       [307., 492., 195.],
       [121., 427., 196.],
       [327.,  45., 181.],
       [423., 316., 181.],
       [235., 102., 180.],
       [416., 106., 190.],
       [ 29., 115., 190.],
       [208.,  96., 177.],
       [ 81., 166., 195.],
       [ 48.,  39., 196.],
       [476., 254., 193.],
       [420., 500., 196.],
       [428., 347., 193.],
       [ 45., 264., 180.],
       [212., 111., 193.],
       [482., 482., 190.],
       [254., 199., 190.],
 

In [25]:
import pandas as pd

gt_df = pd.read_csv(
    "particle_locations.txt",
    delim_whitespace=True,
    header=None,
    names=["label","x","y","z","c4","c5","c6"]
)

gt_5mrc = gt_df[gt_df["label"] == "5MRC"]

gt_xyz = gt_5mrc[["x","y","z"]].to_numpy(float)

print("GT count:", len(gt_xyz))
print("GT min/max:", gt_xyz.min(), gt_xyz.max())

GT count: 131
GT min/max: 19.0 491.0


/tmp/ipykernel_14301/1345508827.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  gt_df = pd.read_csv(


In [24]:
from scipy.spatial import cKDTree

def evaluate(detected, gt, max_dist=5):
    tree = cKDTree(gt)
    dists, _ = tree.query(detected, k=1)
    hits = np.sum(dists <= max_dist)
    return hits, dists

hits_pytom, d_pytom = evaluate(pytom_xyz_vox, gt_xyz)
hits_ri, d_ri = evaluate(ri_xyz_vox, gt_xyz)
hits_std, d_std = evaluate(std_xyz_vox, gt_xyz)

print("PyTom hits:", hits_pytom)
print("RI hits:", hits_ri)
print("STD hits:", hits_std)

PyTom hits: 0
RI hits: 0
STD hits: 0


In [27]:
print("PyTom voxel min/max per axis:")
print("X:", pytom_xyz_vox[:,0].min(), pytom_xyz_vox[:,0].max())
print("Y:", pytom_xyz_vox[:,1].min(), pytom_xyz_vox[:,1].max())
print("Z:", pytom_xyz_vox[:,2].min(), pytom_xyz_vox[:,2].max())

print("\nGT min/max per axis:")
print("X:", gt_xyz[:,0].min(), gt_xyz[:,0].max())
print("Y:", gt_xyz[:,1].min(), gt_xyz[:,1].max())
print("Z:", gt_xyz[:,2].min(), gt_xyz[:,2].max())

PyTom voxel min/max per axis:
X: 25.0 496.0
Y: 20.0 502.0
Z: 173.0 196.0

GT min/max per axis:
X: 22.0 491.0
Y: 19.0 487.0
Z: 21.0 160.0


In [31]:
gt_centered_vox = gt_xyz_raw - 256
# gt_ang = gt_centered * 15
# gt_xyz_vox_corrected = (gt_ang / 15) + 256

In [32]:
pytom_centered_vox = pytom_xyz_vox - 256

In [33]:
hits_pytom, _ = evaluate(pytom_centered_vox, gt_centered_vox)
hits_ri, _    = evaluate(ri_xyz_vox - 256, gt_centered_vox)
hits_std, _   = evaluate(std_xyz_vox - 256, gt_centered_vox)

print("PyTom hits:", hits_pytom)
print("RI hits:", hits_ri)
print("STD hits:", hits_std)

PyTom hits: 0
RI hits: 0
STD hits: 0


In [34]:
print("GT centered Z min/max:", 
      gt_centered_vox[:,2].min(), 
      gt_centered_vox[:,2].max())

print("PyTom centered Z min/max:", 
      pytom_centered_vox[:,2].min(), 
      pytom_centered_vox[:,2].max())

GT centered Z min/max: -235.0 -96.0
PyTom centered Z min/max: -83.0 -60.0


In [35]:
print("Median GT Z:", np.median(gt_centered_vox[:,2]))
print("Median PyTom Z:", np.median(pytom_centered_vox[:,2]))

Median GT Z: -162.0
Median PyTom Z: -67.0


In [36]:
z_shift = np.median(pytom_centered_vox[:,2]) - np.median(gt_centered_vox[:,2])
print("Estimated Z shift:", z_shift)

Estimated Z shift: 95.0


In [37]:
gt_corrected = gt_centered_vox.copy()
gt_corrected[:,2] += z_shift

In [38]:
hits_pytom, _ = evaluate(pytom_centered_vox, gt_corrected)
hits_ri, _    = evaluate(ri_xyz_vox - 256, gt_corrected)
hits_std, _   = evaluate(std_xyz_vox - 256, gt_corrected)

print("PyTom hits:", hits_pytom)
print("RI hits:", hits_ri)
print("STD hits:", hits_std)

PyTom hits: 0
RI hits: 0
STD hits: 0


In [40]:
from scipy.spatial import cKDTree
import numpy as np

def evaluate_xy(detected, gt, max_dist=5):
    tree = cKDTree(gt[:, :2])        
    dists, _ = tree.query(detected[:, :2], k=1)
    hits = np.sum(dists <= max_dist)
    return hits, dists

# Use voxel coordinates (not centered)
hits_pytom_xy, _ = evaluate_xy(pytom_xyz_vox, gt_xyz_raw, max_dist=5)
hits_ri_xy, _    = evaluate_xy(ri_xyz_vox, gt_xyz_raw, max_dist=5)
hits_std_xy, _   = evaluate_xy(std_xyz_vox, gt_xyz_raw, max_dist=5)

print("2D hits (ignore Z)")
print("PyTom:", hits_pytom_xy)
print("RI:", hits_ri_xy)
print("STD:", hits_std_xy)

2D hits (ignore Z)
PyTom: 5
RI: 46
STD: 21


In [41]:
import starfile
import numpy as np

voxel_size = 15.0
center = 512 / 2  # 256

df_pytom = starfile.read("reconstruction_fixed_particles_pytom_150.star")

pytom_ang = df_pytom[
    ["rlnCenteredCoordinateXAngst",
     "rlnCenteredCoordinateYAngst",
     "rlnCenteredCoordinateZAngst"]
].to_numpy(float)

pytom_vox = pytom_ang / voxel_size + center

print("PyTom voxel min/max per axis:")
print("X:", pytom_vox[:,0].min(), pytom_vox[:,0].max())
print("Y:", pytom_vox[:,1].min(), pytom_vox[:,1].max())
print("Z:", pytom_vox[:,2].min(), pytom_vox[:,2].max())

PyTom voxel min/max per axis:
X: 12.0 504.0
Y: 13.0 500.0
Z: 174.0 339.0


In [42]:
import pandas as pd

gt = pd.read_csv(
    "particle_locations.txt",
    sep=r"\s+",
    header=None,
    names=["label", "z", "y", "x", "rot1", "rot2", "rot3"]
)

gt = gt[~gt["label"].isin(["vesicle", "fiducial"])]
gt = gt[gt["label"] == "5MRC"]

gt_vox = gt[["x", "y", "z"]].to_numpy(float)

print("GT voxel min/max per axis:")
print("X:", gt_vox[:,0].min(), gt_vox[:,0].max())
print("Y:", gt_vox[:,1].min(), gt_vox[:,1].max())
print("Z:", gt_vox[:,2].min(), gt_vox[:,2].max())
print("GT count:", len(gt_vox))

GT voxel min/max per axis:
X: 21.0 160.0
Y: 19.0 487.0
Z: 22.0 491.0
GT count: 131


In [43]:
from scipy.spatial import cKDTree

def count_hits(pred, gt, max_dist=3):
    tree = cKDTree(gt)
    dists, _ = tree.query(pred, k=1)
    return int((dists < max_dist).sum()), dists

hits, dists = count_hits(pytom_vox, gt_vox, max_dist=3)

print("PyTom hits:", hits, "/", len(pytom_vox))
print("Median nearest distance:", float(np.median(dists)))
print("Mean nearest distance:", float(np.mean(dists)))

PyTom hits: 0 / 150
Median nearest distance: 128.42084000128028
Mean nearest distance: 152.37477315542236


In [67]:
import numpy as np
import pandas as pd
import starfile
from itertools import permutations, product
from scipy.spatial import cKDTree

VOX = 15.0
CENTER = 256.0
N = 512  # volume size

# ---- load pytom star -> vox ----
df_pytom = starfile.read("reconstruction_fixed_particles_pytom_1500.star")
pytom_ang = df_pytom[[
    "rlnCenteredCoordinateXAngst",
    "rlnCenteredCoordinateYAngst",
    "rlnCenteredCoordinateZAngst"
]].to_numpy(float)
pytom_vox = pytom_ang / VOX + CENTER

# ---- load GT text (you can change these names if needed) ----
gt = pd.read_csv(
    "particle_locations.txt",
    sep=r"\s+",
    header=None,
    names=["label", "c1", "c2", "c3", "rot1", "rot2", "rot3"]
)

# keep only 5MRC protein points (your mentor’s filter style)
gt = gt[~gt["label"].isin(["vesicle", "fiducial"])]
gt = gt[gt["label"] == "5MRC"]

gt_raw = gt[["c1","c2","c3"]].to_numpy(float)

def hits(pred, gt, max_dist=3.0):
    tree = cKDTree(gt)
    dists, _ = tree.query(pred, k=1)
    return int((dists < max_dist).sum()), float(np.median(dists))

best = None

# perm = how to interpret GT columns as (X,Y,Z)
# flip mask = whether to flip each axis around volume
for perm in permutations([0,1,2]):  # mapping to (X,Y,Z)
    gt_perm = gt_raw[:, perm].copy()

    for flips in product([False, True], repeat=3):
        gt_test = gt_perm.copy()
        for ax, doflip in enumerate(flips):
            if doflip:
                gt_test[:, ax] = (N - 1) - gt_test[:, ax]

        h, med = hits(pytom_vox, gt_test, max_dist=3.0)

        cand = (h, med, perm, flips,
                gt_test.min(axis=0), gt_test.max(axis=0))
        if best is None or cand[0] > best[0] or (cand[0] == best[0] and cand[1] < best[1]):
            best = cand

h, med, perm, flips, mn, mx = best
print("BEST mapping")
print("hits:", h, "/", len(pytom_vox), "  median NN dist:", med)
print("perm (GT c1,c2,c3 -> X,Y,Z):", perm)
print("flips (X,Y,Z):", flips)
print("GT mapped min:", mn, "max:", mx)
print("PyTom min:", pytom_vox.min(axis=0), "max:", pytom_vox.max(axis=0))

BEST mapping
hits: 1 / 541   median NN dist: 120.25805586321442
perm (GT c1,c2,c3 -> X,Y,Z): (2, 1, 0)
flips (X,Y,Z): (False, True, True)
GT mapped min: [21. 24. 20.] max: [160. 492. 489.]
PyTom min: [  8.  11. 174.] max: [504. 502. 339.]


In [68]:
z_min, z_max = pytom_vox[:,2].min(), pytom_vox[:,2].max()

gt_filtered = gt_raw.copy()

# First apply correct permutation (from brute force result)
perm = (1,2,0)
gt_perm = gt_filtered[:, perm]

# Now filter by Z overlap
mask = (gt_perm[:,2] >= z_min) & (gt_perm[:,2] <= z_max)
gt_overlap = gt_perm[mask]

print("Original GT:", len(gt_raw))
print("GT inside PyTom Z slab:", len(gt_overlap))

Original GT: 131
GT inside PyTom Z slab: 54


In [69]:
from scipy.spatial import cKDTree

tree = cKDTree(gt_overlap)
dists, _ = tree.query(pytom_vox, k=1)

hits = (dists < 3).sum()
print("PyTom hits:", hits, "/", len(gt_overlap))
print("Median NN distance:", np.median(dists))

PyTom hits: 0 / 54
Median NN distance: 127.72626981165621


In [70]:
print("PyTom voxel ranges:")
print(pytom_xyz_vox.min(axis=0), pytom_xyz_vox.max(axis=0))

print("GT voxel ranges:")
print(gt_xyz.min(axis=0), gt_xyz.max(axis=0))

PyTom voxel ranges:
[ 25.  20. 173.] [496. 502. 196.]
GT voxel ranges:
[21. 19. 22.] [160. 487. 491.]


In [71]:
import numpy as np
from scipy.spatial import cKDTree

def hits_2d(pred_xyz, gt_xyz, max_dist=10):
    # pred_xyz, gt_xyz are voxel coords (N,3)
    pred_xy = pred_xyz[:, :2]  # X,Y
    gt_xy   = gt_xyz[:, :2]
    tree = cKDTree(gt_xy)
    d, idx = tree.query(pred_xy, k=1)
    hits = np.where(d <= max_dist)[0]
    return d, hits

# Example usage:
d_p, hits_p = hits_2d(pytom_xyz_vox, gt_xyz, max_dist=10)
print("PyTom 2D hits:", len(hits_p), "/", len(pytom_xyz_vox), "median dist:", np.median(d_p))

d_r, hits_r = hits_2d(ri_xyz_vox, gt_xyz, max_dist=10)
print("RI 2D hits:", len(hits_r), "/", len(ri_xyz_vox), "median dist:", np.median(d_r))

d_s, hits_s = hits_2d(std_xyz_vox, gt_xyz, max_dist=10)
print("STD 2D hits:", len(hits_s), "/", len(std_xyz_vox), "median dist:", np.median(d_s))

PyTom 2D hits: 35 / 137 median dist: 76.32168761236873
RI 2D hits: 41 / 150 median dist: 34.76197164747201
STD 2D hits: 55 / 150 median dist: 18.055385138137417


In [76]:
import numpy as np
import pandas as pd
import starfile
from scipy.spatial import cKDTree

# -----------------------
# Known constants
# -----------------------

VOXEL_SIZE = 15.0
VOL_SHAPE = np.array([512,512,512])  # Z,Y,X
CENTER = np.array([512/2,512/2,512/2])  # X,Y,Z


# -----------------------
# Load PyTom star
# -----------------------

df = starfile.read("reconstruction_fixed_particles_pytom_1500.star")

pytom_ang = df[
    ["rlnCenteredCoordinateXAngst",
     "rlnCenteredCoordinateYAngst",
     "rlnCenteredCoordinateZAngst"]
].to_numpy(float)

# Convert Å → voxel (centered → absolute)
pytom_vox = pytom_ang / VOXEL_SIZE + CENTER


print("PyTom voxel min:", pytom_vox.min(axis=0))
print("PyTom voxel max:", pytom_vox.max(axis=0))


# -----------------------
# Load GT
# -----------------------

gt = pd.read_csv(
    "particle_locations.txt",
    sep=r"\s+",
    header=None,
    names=["label","z","y","x","r1","r2","r3"]
)

gt = gt[gt["label"]=="5MRC"]
gt_xyz = gt[["x","y","z"]].to_numpy(float)

print("GT voxel min:", gt_xyz.min(axis=0))
print("GT voxel max:", gt_xyz.max(axis=0))
print("GT count:", len(gt_xyz))


# -----------------------
# Distance test
# -----------------------

tree = cKDTree(gt_xyz)
dists, _ = tree.query(pytom_vox, k=1)

print("\nMedian NN distance:", np.median(dists))
print("Mean NN distance:", np.mean(dists))
print("Hits (dist < 5 vox):", np.sum(dists < 41))

PyTom voxel min: [  8.  11. 174.]
PyTom voxel max: [504. 502. 339.]
GT voxel min: [21. 19. 22.]
GT voxel max: [160. 487. 491.]
GT count: 131

Median NN distance: 123.47469376354006
Mean NN distance: 148.2654737218582
Hits (dist < 5 vox): 124


In [64]:
z_min = pytom_vox[:,2].min()
z_max = pytom_vox[:,2].max()

gt_filtered = gt_xyz[
    (gt_xyz[:,2] >= z_min) &
    (gt_xyz[:,2] <= z_max)
]

print("Filtered GT count:", len(gt_filtered))

tree = cKDTree(gt_filtered)
dists, _ = tree.query(pytom_vox)

print("Hits < 5 vox:", np.sum(dists < 5))
print("Median NN:", np.median(dists))

Filtered GT count: 54
Hits < 5 vox: 3
Median NN: 123.47469376354006


In [77]:
from scipy.spatial import cKDTree
import numpy as np

tree = cKDTree(gt_xyz)

dists, idxs = tree.query(pytom_vox)

threshold = 5

used_gt = set()
hits = 0

for d, i in sorted(zip(dists, idxs), key=lambda x: x[0]):
    if d < threshold and i not in used_gt:
        used_gt.add(i)
        hits += 1

print("Unique GT hits:", hits)
print("GT count:", len(gt_xyz))

Unique GT hits: 3
GT count: 131


In [78]:
from scipy.spatial import cKDTree
import numpy as np

# Use only X,Y
gt_xy = gt_xyz[:, :2]
pytom_xy = pytom_vox[:, :2]

tree = cKDTree(gt_xy)
dists, idxs = tree.query(pytom_xy)

threshold = 5  # voxels in XY plane

used_gt = set()
hits = 0

for d, i in sorted(zip(dists, idxs), key=lambda x: x[0]):
    if d < threshold and i not in used_gt:
        used_gt.add(i)
        hits += 1

print("Unique 2D hits:", hits)
print("GT count:", len(gt_xy))
print("2D Recall:", hits / len(gt_xy))

Unique 2D hits: 18
GT count: 131
2D Recall: 0.13740458015267176


In [80]:
import pandas as pd
import numpy as np

# Load GT
gt = pd.read_csv(
    "particle_locations.txt",
    sep=r"\s+",
    header=None,
    names=["label", "z", "y", "x", "rot1", "rot2", "rot3"]
)

# Keep only 5MRC
gt = gt[gt["label"] == "5MRC"]

# GT is already in voxel coordinates
gt_xyz_vox = gt[["x", "y", "z"]].to_numpy(float)

print("GT voxel min:", gt_xyz_vox.min(axis=0))
print("GT voxel max:", gt_xyz_vox.max(axis=0))
print("GT count:", len(gt_xyz_vox))

GT voxel min: [21. 19. 22.]
GT voxel max: [160. 487. 491.]
GT count: 131


In [82]:
voxel_size = 15.0
center = 256

pytom_xyz_vox = (pytom_xyz_ang / voxel_size) + center
print("PyTom voxel min:", pytom_xyz_vox.min(axis=0))
print("PyTom voxel max:", pytom_xyz_vox.max(axis=0))

PyTom voxel min: [ 25.  20. 173.]
PyTom voxel max: [496. 502. 196.]


In [ ]:
from scipy.spatial import cKDTree

scale = 1.5  # test 15/10 hypothesis
center = 256

# scale around center
pytom_scaled = (pytom_xyz_vox - center) * scale + center

tree = cKDTree(gt_xyz_vox)
dists, _ = tree.query(pytom_scaled)

print("After scaling:")
print("Median NN:", np.median(dists))
print("Mean NN:", np.mean(dists))
print("Hits (<5 vox):", np.sum(dists < 5))

After scaling:
Median NN: 104.76756940962218
Mean NN: 116.32321721633717
Hits (<5 vox): 0


In [85]:
scales = np.linspace(0.5, 2.0, 60)

best_hits = 0
best_scale = None

tree = cKDTree(gt_xyz_vox)

for s in scales:
    test = (pytom_xyz_vox - center) * s + center
    d, _ = tree.query(test)
    hits = np.sum(d < 5)

    if hits > best_hits:
        best_hits = hits
        best_scale = s

print("Best scale:", best_scale)
print("Best hits:", best_hits)

Best scale: 0.9830508474576272
Best hits: 2


In [86]:
import numpy as np

voxel_size = 15.0
center = 256.0

# pick a few points
p = pytom_xyz_vox[:10].copy()

# vox -> ang -> vox
ang = (p - center) * voxel_size
vox_back = ang / voxel_size + center

err = np.linalg.norm(vox_back - p, axis=1)
print("Roundtrip error (should be ~0):")
print("max:", err.max(), "mean:", err.mean())

Roundtrip error (should be ~0):
max: 0.0 mean: 0.0


In [87]:
import numpy as np
from itertools import permutations, product
from scipy.spatial import cKDTree

tree = cKDTree(gt_xyz_vox)

center = 256.0
thr = 5.0

best = None
best_hits = -1

P = pytom_xyz_vox.copy()

for perm in permutations([0,1,2]):          # axis swaps
    Pp = P[:, perm]
    for signs in product([1,-1], repeat=3): # sign flips
        s = np.array(signs, float)
        # flip around center
        Pt = (Pp - center) * s + center

        d, _ = tree.query(Pt)
        hits = np.sum(d < thr)

        if hits > best_hits:
            best_hits = hits
            best = (perm, signs, np.median(d), np.mean(d))

print("Best transform:")
print("perm (xyz indices):", best[0], "signs:", best[1])
print("hits<thr:", best_hits, "median NN:", best[2], "mean NN:", best[3])

Best transform:
perm (xyz indices): (0, 1, 2) signs: (1, 1, 1)
hits<thr: 1 median NN: 96.5090669315583 mean NN: 131.99639970132748


In [88]:
p_center = (pytom_xyz_vox.min(axis=0) + pytom_xyz_vox.max(axis=0)) / 2
g_center = (gt_xyz_vox.min(axis=0) + gt_xyz_vox.max(axis=0)) / 2
print("PyTom inferred center:", p_center)
print("GT inferred center:", g_center)

PyTom inferred center: [260.5 261.  184.5]
GT inferred center: [ 90.5 253.  256.5]


In [89]:
print("PyTom z range:", pytom_xyz_vox[:,2].min(), pytom_xyz_vox[:,2].max())
print("GT z range:", gt_xyz_vox[:,2].min(), gt_xyz_vox[:,2].max())

# How many GT points fall inside PyTom z slab?
zmin, zmax = pytom_xyz_vox[:,2].min(), pytom_xyz_vox[:,2].max()
gt_in_slab = gt_xyz_vox[(gt_xyz_vox[:,2] >= zmin) & (gt_xyz_vox[:,2] <= zmax)]
print("GT in PyTom z-slab:", len(gt_in_slab), "/", len(gt_xyz_vox))

PyTom z range: 173.0 196.0
GT z range: 22.0 491.0
GT in PyTom z-slab: 5 / 131


In [90]:
import numpy as np
from scipy.spatial import cKDTree

pmin = pytom_xyz_vox.min(axis=0)
pmax = pytom_xyz_vox.max(axis=0)

gt_f = gt_xyz_vox[
    (gt_xyz_vox[:,0] >= pmin[0]) & (gt_xyz_vox[:,0] <= pmax[0]) &
    (gt_xyz_vox[:,1] >= pmin[1]) & (gt_xyz_vox[:,1] <= pmax[1]) &
    (gt_xyz_vox[:,2] >= pmin[2]) & (gt_xyz_vox[:,2] <= pmax[2])
]

print("GT comparable (inside PyTom bbox):", len(gt_f), "/", len(gt_xyz_vox))

tree = cKDTree(gt_f)
d, idx = tree.query(pytom_xyz_vox)
thr = 5.0
hits = np.sum(d < thr)
unique_gt = len(np.unique(idx[d < thr]))

print("Hits < 5 vox:", hits)
print("Unique GT hits:", unique_gt)

GT comparable (inside PyTom bbox): 5 / 131
Hits < 5 vox: 1
Unique GT hits: 1


In [98]:
from scipy.spatial import cKDTree
import numpy as np

thr_xy = 400.0

p_xy = pytom_xyz_vox[:, :2]
gt_xy = gt_xyz_vox[:, :2]

tree2 = cKDTree(gt_xy)
d2, idx2 = tree2.query(p_xy)

hits2 = np.sum(d2 < thr_xy)
unique_gt2 = len(np.unique(idx2[d2 < thr_xy]))

print("2D hits < thr:", hits2)
print("Unique 2D GT hits:", unique_gt2)
print("2D recall:", unique_gt2 / len(gt_xyz_vox))

2D hits < thr: 137
Unique 2D GT hits: 44
2D recall: 0.33587786259541985


In [99]:
voxel_size = 15.0

p_vox = pytom_xyz_vox[:5].copy()
p_ang = p_vox * voxel_size

print("Example point vox:", p_vox[0])
print("Converted to Å:", p_ang[0])
print("Ratio check (should be ~15):", p_ang[0] / p_vox[0])

Example point vox: [ 60. 403. 189.]
Converted to Å: [ 900. 6045. 2835.]
Ratio check (should be ~15): [15. 15. 15.]
